In [4]:
import math
import numpy as np
from itertools import combinations

# algorithm for generating bits through poor man's dedekind cut.

In [29]:
def f(num,denom):
    return math.ceil(abs(math.log2(num) - math.log2(denom)))

def logexpand(b, M): # generate bits of 1/b
    assert(b > 1)
    assert(M > 0 and type(M) == int)
    s = [-1]
    nums = [-1]
    denoms = [-1]
    
    denoms.append(b)
    nums.append(1)
    s.append(f(1,b))
    
    denoms.append((2 ** s[-1])*b)
    nums.append((2 ** s[-1]) - b)
    s.append(f(nums[-1], (2 ** s[-1])*b))
    
    while(True):
        denomsi = (2 ** sum(s[1:]))*b
        numsi = (2 ** sum(s[1:])) - b*sum([2**(sum(comb)) for comb in list(combinations(s[1:], len(s[1:])-1))])
        denoms.append(denomsi)
        nums.append(numsi)
        
        #print(denoms[1:])
        #print(nums[1:])
        #print("before",s[1:])
        s.append(f(nums[-1], denoms[-1]))
        if(s[-1] > M):
            s.pop(-1)
            break
    
    
    return (nums[1:], denoms[1:], s[1:])
    

# basic function to view whats happening each step of the algo

In [30]:
def test(b,M,flag):
    nums, denoms, s = logexpand(b, M)
    if(flag):
        print("nums",nums,"\n")
        print("denoms",denoms,"\n")
    
    print("exponents, treat as negative",s,"\n")
    #print("bitstring")
    bitstring = "0."
    
    # retains order since the exponents come sorted
    for bit in  range(1,M):
        if bit in s:
            bitstring += "1"
        else:
            bitstring += "0"
    
    print("bitstring",bitstring,"\n")
    
    print("number of bits needed for each iterations numerator to then do log2 on.  Shows its unfeasible")
    for num in nums[1:]:
        print(math.log2(num))
        
    print("actual", 1/b)
    print("estimate", sum([(2 ** -exponent) for exponent in s]))
    return s
    

# output of algorithm for 1/9 using 32 bits

In [31]:
test(9,32,True)
#test(2**4)

nums [1, 7, 80, 512, 229376, 167772160, 68719476736, 1970324836974592, 92233720368547758080, 2417851639229258349412352, 4436777100798802905238461218816, 13292279957849158729038070602803445760, 22300745198530623141535718272648361505980416, 2619010934096978029421003220227579171223431117012992, 502168138830934461106863153856613313288188435557122761031680, 53919893334301279589334030174039261347274288845081144962207220498432] 

denoms [9, 144, 4608, 294912, 301989888, 618475290624, 2533274790395904, 166020696663385964544, 21760664753063325144711168, 5704427701027032306735164424192, 23926103924128485712268527085046202368, 200706706786775608273821464453835253553823744, 3367299772410400323541289854578316077287268579016704, 903902649895682029992353676941903963918739184002820969857024, 485279040008711516304006271566353352125468599605730304659864984485888, 521064401567922879406069432539095585339714930995382538177559128035609083379712] 

exponents, treat as negative [4, 5, 6, 10, 11, 12, 16, 17, 1

[4, 5, 6, 10, 11, 12, 16, 17, 18, 22, 23, 24, 28, 29, 30]

# output of algorithm for 1/8! using 64 bits

In [32]:
fact8 = 1 * 2 * 3 * 4 * 5 * 6 * 7 * 8
test(fact8, 64, True)

nums [1, 25216, 662700032, 1099511627776, 113562768203774427136, 12224657887943130214628851712, 83076749736557242056487941267521536, 35145974432884262071060291997693817733425135616, 15496594909235868135719605138543926464752690003520585203712, 431359146674410236714672241392314090778194310760649159697657763987456, 747472210485290287052996085242643016813582318932170870481654384215945594112160301056, 1349943806702015172073653718299156936428909258129404550606530296143642866693364827614512038943916032, 153914086704665934422965000391185991426092731525255651046673021110334850669910978950836977558144201721900890587136, 1092431859483601054673895188552374979153956207698071730617168954762160795275416495477749916005030993770372137118793007300615818182656] 

denoms [40320, 2642411520, 346346162749440, 181585136975578398720, 48743889046861848324153016320, 26169176167015531247793701499269283840, 56197877900297170316670010047073870995070648320, 6179022020771263876900855213470046628350756140644283973632

[16, 17, 19, 28, 29, 31, 40, 41, 43, 52, 53, 55, 64]

# Helper functions to take in a number of bits M and degree of taylor polynomial N, to then use log expansion to construct coefficients of taylor polynomial

In [33]:
def fact(N):
    res = [1, 1]
    for i in range(2,N+1):
        res.append(res[-1] * i)
    
    return res
    

def function_handler(coef_id, N, M):
    if(coef_id == 0):
        args = fact(2*N+1)[1::2]
    #if(coef_id == 1):
    #   args = ...
    
    res = [[0]]
    for arg in args[1:]:
        res.append(test(arg,M,False))
        
    return res
        
def taylorhelper(N,M,func):
    assert(type(N) == type(M) == int)
    assert(type(func) == str)
    functions = [("sin",0), ("cos",1), ("ddxln",2)]
    
    for f in functions:
        if f[0] == func:
            return function_handler(f[1], N, M)
    
    return None

# for log10, loge, etc hardcode an estimate of the change of base coefficient for log2 -> loga
def taylorloga(N,M):
    return

# 0..inf (-1)^k * x^(2k+1) / (2k+1)! # Try to mod arg by 2pi = (2 * 22/7) in p4
def taylorcos(N,M):
    return

# 0..inf (-1)^k * x^(2k+1) / (2k+1)! # Try to mod arg by 2pi = (2 * 22/7) in p4
def taylorsin(N,M):
    res = taylorhelper(N,M,"sin")
    coef = [sum([(2 ** -i) for i in s]) for s in res]
    
    for i in range(len(coef)):
        if(i % 2 == 1):
            coef[i] = -coef[i]
    
    print(coef)
    return lambda x: sum([coef[i] * (x ** (2*i + 1)) for i in range(len(coef))])

# d/dx of ln(1+x) is 1/1+x, which is also the power series version of geometric series
# converges uniformly for |x| < 1.  use this inside of logexpansion to reduce space complexity?
# if so, now we construct a log2 taylor expansion close to 0, and its arguments will be a sequence
# that converges to 0, instead of one that diverges to infinity
# also note, if x = 1/b, then it holds for |1/b| < 1 -> 1 < |b|
def taylorddxln(N,M): 
    return

# Needed recripricols for 8th degree taylor polynomial for sin

In [34]:
# factorials for sin
factorials = fact(2*8)
factorials[1::2]

[1, 6, 120, 5040, 362880, 39916800, 6227020800, 1307674368000]

# retrieve lambda for sin, construct unit circle, and evaluate

In [62]:
sin = taylorsin(8,56)

#pi = math.pi
pi = 22/7
#points = [pi/6, pi/4, pi/3, pi/3, 2*pi/3,
#          3*pi/4, 5*pi/6, pi, 7*pi/6, 5*pi/4,
#          4*pi/3, 3*pi/2, 5*pi/3, 7*pi/4, 11*pi/6, 0]
points = np.linspace(0, 2*math.pi, 360)

exponents, treat as negative [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55] 

bitstring 0.0010101010101010101010101010101010101010101010101010101 

number of bits needed for each iterations numerator to then do log2 on.  Shows its unfeasible
1.0
4.0
9.0
16.0
25.0
36.0
49.0
64.0
81.0
100.0
121.0
144.0
169.0
196.0
225.0
256.0
289.0
324.0
361.0
400.0
441.0
484.0
529.0
576.0
625.0
676.0
729.0
actual 0.16666666666666666
estimate 0.16666666666666666
exponents, treat as negative [7, 11, 15, 19, 23, 27, 31, 35, 39, 43, 47, 51, 55] 

bitstring 0.0000001000100010001000100010001000100010001000100010001 

number of bits needed for each iterations numerator to then do log2 on.  Shows its unfeasible
3.0
10.0
21.0
36.0
55.0
78.0
105.0
136.0
171.0
210.0
253.0
300.0
351.0
actual 0.008333333333333333
estimate 0.008333333333333331
exponents, treat as negative [13, 14, 16, 25, 26, 28, 37, 38, 40, 49, 50, 52] 

bitstring 0.00000000000011010000000011

In [63]:
points

array([0.        , 0.01750191, 0.03500382, 0.05250573, 0.07000764,
       0.08750954, 0.10501145, 0.12251336, 0.14001527, 0.15751718,
       0.17501909, 0.192521  , 0.21002291, 0.22752482, 0.24502673,
       0.26252863, 0.28003054, 0.29753245, 0.31503436, 0.33253627,
       0.35003818, 0.36754009, 0.385042  , 0.40254391, 0.42004581,
       0.43754772, 0.45504963, 0.47255154, 0.49005345, 0.50755536,
       0.52505727, 0.54255918, 0.56006109, 0.57756299, 0.5950649 ,
       0.61256681, 0.63006872, 0.64757063, 0.66507254, 0.68257445,
       0.70007636, 0.71757827, 0.73508018, 0.75258208, 0.77008399,
       0.7875859 , 0.80508781, 0.82258972, 0.84009163, 0.85759354,
       0.87509545, 0.89259736, 0.91009926, 0.92760117, 0.94510308,
       0.96260499, 0.9801069 , 0.99760881, 1.01511072, 1.03261263,
       1.05011454, 1.06761644, 1.08511835, 1.10262026, 1.12012217,
       1.13762408, 1.15512599, 1.1726279 , 1.19012981, 1.20763172,
       1.22513363, 1.24263553, 1.26013744, 1.27763935, 1.29514

# Basic measurements

In [64]:
actual = [math.sin(x) for x in points]
estimate = [sin(x) for x in points]
diff = [abs(x - y) for x,y in list(zip(actual,estimate))]
diff

[0.0,
 3.469446951953614e-18,
 0.0,
 6.938893903907228e-18,
 0.0,
 0.0,
 0.0,
 1.3877787807814457e-17,
 0.0,
 0.0,
 2.7755575615628914e-17,
 2.7755575615628914e-17,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.551115123125783e-17,
 0.0,
 5.551115123125783e-17,
 0.0,
 5.551115123125783e-17,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.551115123125783e-17,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.1102230246251565e-16,
 0.0,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 0.0,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 0.0,
 1.1102230246251565e-16,
 0.0,
 0.0,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 0.0,
 0.0,
 0.0,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 1.1102230246251565e-16,
 0.0,
 0.0,
 1.1102230246251565e-16,
 2.220446049250313e-16,
 2.220446049250313

In [65]:
print("max absolute difference", max(diff))
diff = np.array(diff)
print("average absolute difference", np.mean(diff))
print("with standard deviation", np.var(diff) ** 0.5)

max absolute difference 0.010689386006526769
average absolute difference 0.0005508407554925449
with standard deviation 0.0016725472002560742


In [66]:
abs(math.pi - pi)

0.0012644892673496777